In [ ]:
import sys
import os

In [ ]:
script_dir = os.getcwd()

In [ ]:
script_dir

In [ ]:
sys.path.append(os.path.join(script_dir, '../'))

### Set GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jovyan/.config/gcloud/application_default_credentials.json'

In [ ]:
from monitor import Monitor

### Config dic example

In [ ]:
config = {
    "fetchers":[
        {
            "type": "BQ_DATA_FETCHER",
            "name": "count_requests",
            "sql": """
                SELECT event_date, source_system, os, sum(count) as requests 
                FROM your_project.your_dataset.metrics_requests
                WHERE event_date = {execution_date}
                GROUP BY 1, 2, 3
            """,
            "intervals": ["1 DAY"]
        },
        {
            "type": "BQ_DATA_FETCHER",
            "name": "count_sessions",
            "sql": """
                SELECT PROCESSED_DATE, count(1) as sessions 
                FROM your_project.your_dataset.metrics_sessions
                WHERE PROCESSED_DATE = {execution_date}
                GROUP BY 1
            """,
            "intervals": ["1 DAY", "1 WEEK"]
        }
    ],
    "rules": [
        {
            "name": "known os types",
            "type": "CROSS_SECTION",
            "spec": {
                "metric": ["count_requests.os"],
                "function": "IN",
                "list": ['ios', 'other', 'amx', 'android']
            },
            "fail_task": True,
        },
        {
            "name": "request above",
            "type": "CROSS_SECTION",
            "spec": {
                "metric": ["count_requests.requests", 0],
                "function": "GREATER_THAN", # abs(METRIC_X - METRIC_Y) / METRIC_X > 0.2
                "threshold": 0.2
            },
            "fail_task": True,
        },
        {
            "name": "request above oposite",
            "type": "CROSS_SECTION",
            "spec": {
                "metric": [100, "count_requests.requests"],
                "function": "GREATER_THAN", # abs(METRIC_X - METRIC_Y) / METRIC_X > 0.2
                "threshold": 0.2
            },
            "fail_task": True,
        },
        {
            "name": "request above threshold",
            "type": "CROSS_SECTION",
            "spec": {
                "metric": ["count_requests.requests", 100],
                "function": "RELATIVE_GREATER_THAN", # abs(METRIC_X - METRIC_Y) / METRIC_X > 0.2
                "threshold": 0.2
            },
            "fail_task": True,
        },
        {
            "name": "request above threshold oposite",
            "type": "CROSS_SECTION",
            "spec": {
                "metric": [100, "count_requests.requests"],
                "function": "RELATIVE_GREATER_THAN", # abs(METRIC_X - METRIC_Y) / METRIC_X > 0.2
                "threshold": 0.2
            },
            "fail_task": True,
        },
        {
            "name": "request above THRESHOLD",
            "type": "TIME_SERIES",
            "spec": {
                "metric": "count_sessions.sessions",
                "threshold": 0.2,
                "function": "RELATIVE_GREATER_THAN" #greater than
            }
        },
        {
            "name": "request above threshold 121212",
            "type": "TIME_SERIES", 
            "spec": {
                "metric": "count_requests.requests",
                "function": "RELATIVE_GREATER_THAN", #relative greater than
                "threshold": 0.20,
                "dimensions": ['source_system', 'os']
            },
            "fail_task": True,
        }
    ],
    "actions":[
        {
            "type": "PRINT",
            "address":['your_email@gmail.com'],
            "always_send": True
        }
    ]
}

In [ ]:
monitor = Monitor(config, 'gcp-project', {'execution_date':"'2022-05-01'", 'task_id':'metrics canary'})

In [ ]:
fetcher_map = monitor.run()

### Config file example

In [ ]:
monitor = Monitor('/home/jovyan/repos/signal/sample/checker_config.json', 'gcp-project-id', {'execution_date':"'2022-05-01'", 'task_id':'metrics canary'})

In [ ]:
fetcher_map = monitor.run()